### Load tsv files and combine them into a single dataframe 

In [36]:
import pandas as pd
import os

# Specify the folder containing the CSV files  
folder_path = '/Users/apoltavets/anna-apps/annalyticai/careear-stats/output/job_posting_details/'
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.tsv')] 
dataframes = [] 
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep='\t')
    dataframes.append(df)
    
combined_df = pd.concat(dataframes, ignore_index=True)
print(combined_df.shape)
combined_df.drop_duplicates(subset=['linkedin_job_id'], inplace=True)
print(combined_df.shape)
combined_df = combined_df[combined_df['job_title'].notna()]
combined_df.drop_duplicates(subset=['organization_name', 'payer_name', 'job_title', 'job_description'], inplace=True)
print(combined_df.shape)

(30777, 9)
(13956, 9)
(7661, 9)


/var/folders/x8/ps_xddr95734lgrb6gqpr7g80000gp/T/ipykernel_57864/315580819.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dataframes, ignore_index=True)


### Clean up job titles and classify them as IT or non-IT

In [95]:
import re

def title_cleanup(job_title):
    if not isinstance(job_title, str):  # Check if the value is not a string
        return job_title

    # Remove text in parentheses
    tmp = re.sub(r'\(.*?\)', '', job_title).strip()
    # Remove numbers at the beginning
    tmp = re.sub(r'^\d+\s*', '', tmp).strip()
    tmp = re.sub(r'^\d+\s*', '', tmp).strip()
    # Remove numbers at the end
    tmp = re.sub(r'\s*\d+$', '', tmp).strip()
    # Additional cleanup
    tmp = re.sub(r'-\s*\d+[,\d]*/\w+\s+\w+$', '', tmp).strip()
    tmp = tmp.replace('Sr.', '').replace('Sr ', '').replace('Senior ', '').replace('Entry Level', '').replace('Internship', '').replace('Intern', '')
    tmp = tmp.replace('Jr.', '').replace('Jr ', '').replace('Junior ', '').replace('Mid-Level ', '').replace('Middle ', '').replace('New Grad', '').replace('Graduate ', '')
    tmp = tmp.replace('Staff ', '').replace('Lead ', '').replace('Principal ', '')
    tmp = tmp.replace('*', '').replace('#', '').replace(' - ', ' ').replace('-', ' ').replace('/', ' ').replace('\\', ' ').replace(',', ' ').replace('.', ' ').replace('&', ' and ')
    tmp = re.sub(r'\s+', ' ', tmp).strip()  # Replace multiple spaces with a single space
    tmp = tmp.replace('Front-End', 'Frontend').replace('Front-end', 'Frontend').replace('.Net', '.NET')
    tmp = tmp.replace('100% remote', '').replace('100% Remote', '').replace('100% REMOTE', '').replace('remote', '').replace('Remote', '').replace('REMOTE', '').strip().strip('-').strip().strip(':').strip()
    return tmp

# Apply the function
combined_df['job_title_upd'] = combined_df['job_title'].apply(title_cleanup)

### Classify job titles as IT or non-IT using pre-trained model

In [96]:
from joblib import load
JOB_TITLE_VECTORIZER_PATH = "models/job_title_vectorizer.joblib"
JOB_TITLE_CLASSIFIER_PATH = "models/job_title_classifier.joblib"
vectorizer = load(JOB_TITLE_VECTORIZER_PATH)
model = load(JOB_TITLE_CLASSIFIER_PATH)

xvec = vectorizer.transform(combined_df['job_title_upd'])
combined_df['is_it'] = model.predict(xvec)
df_it = combined_df[combined_df['is_it'] == 'Yes']
df_it.shape

(4295, 11)

### Classify manually using keywords

In [97]:
mdf = combined_df.copy()
TECH_KEYWORDS = ['software', 'java', '.net', 'php', 'python', 'backend', 'back end', 'back-end', 'frontend', 'front end', 'front-end', 'application', 'web', 'fullstack', 'full stack', 'full-stack', 'c++', 'c#', 'c sharp', 'ruby', 'rails', 'javascript', 'js', 'react', 'angular', 'node', 'devops', 'cloud', 'aws', 'azure', 'gcp', 'docker', 'kubernetes', 'k8s', 'terraform', 'ci/cd', 'microservices', 'android', 'ios', 'mobile', 'flutter', 'swift', 'objective-c', 'objective c', 'scala', 'go', 'golang', 'haskell', 'perl', 'lua', 'rust', 'ui', 'ux', 'unity', 'computer vision', 'c developer', 'c engineer', 'c programmer', 'c# developer', 'c# engineer', 'c# programmer', 'c sharp developer', 'c sharp engineer', 'c sharp programmer', 'c++ developer', 'c++ engineer', 'c++ programmer', 'oracle', 'data', 'database', 'warehouse', 'etl', 'sql', 'power bi', 'devops', 'devsecops', 'ai', 'ml', 'machine learning', 'artificial intelligence', 'qa', 'quality', 'test', 'qc', 'gis', 'web', 'graphic', 'embedded']
TECH_TITLE = ['engineer', 'developer', 'designer', 'architect', 'lead', 'programmer', 'analyst', 'principal', 'scientist', 'administrator', 'specialist', 'manager', 'director', 'tester', 'coder', 'coder', 'principal']

ADDITIONAL_KEYWORDS = ['engineering manager', 'scrum master', 'product owner', 'product manager', 'delivery manager', 'project manager']

def contains_software_keywords(title_lower, keywords):
    return any(keyword in title_lower for keyword in keywords)

def category_classification(title):
    title_lower = title.lower()
    if contains_software_keywords(title_lower, TECH_KEYWORDS) and contains_software_keywords(title_lower, TECH_TITLE):
        return 'IT'
    elif contains_software_keywords(title_lower, ADDITIONAL_KEYWORDS):
        return 'IT'
    else:
        return ''

mdf['category'] = mdf['job_title'].apply(category_classification)
mdf = mdf[mdf['category'] == 'IT']

In [99]:
mdf['job_title_upd'].unique()

array(['Software Engineer', 'Community Director',
       'Equipment Specialist and Facility Coordinator', ...,
       'Solutions Architect Application Architect W2 Contract',
       'Java Cloud Solutions Architect W2 contract',
       'Software Engineer Inference'], shape=(1752,), dtype=object)